# Interactive Evolutionary Algorithm (IEA) for Abstract Images

This notebook implements a **simple interactive evolutionary algorithm** (IEA) where **you** act as the fitness function.

- Each individual is a small abstract image (a grid of colors).
- You rate the images based on how much you like them.
- The algorithm uses your ratings to evolve new generations.

**Instructions**

1. Run the code cell below.
2. You will see a grid of images and sliders for rating.
3. Rate each image from 0 to 5.
4. Click **"Evolve to Next Generation"**.
5. Watch how the population changes over generations.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import (
    VBox, HBox, Button, IntSlider, Output, Layout,
    HTML
)

from IPython.display import display, clear_output

# If needed, in classic Jupyter you can uncomment this:
# %matplotlib inline

# =======================
# Configuration
# =======================

# Image / genome settings
IMG_HEIGHT = 8      # pixels
IMG_WIDTH  = 8      # pixels
CHANNELS   = 3      # RGB

# Evolution settings
POP_SIZE       = 12      # number of individuals per generation
ELITE_COUNT    = 4       # how many top-rated individuals to keep
MUTATION_RATE  = 0.15
MUTATION_STD   = 0.25
MAX_GENERATIONS = 50     # not enforced, just for reference

# UI settings
MAX_RATING = 5           # 0..MAX_RATING


# =======================
# Representation & Operators
# =======================

class Individual:
    def __init__(self, genome=None):
        if genome is None:
            # Random RGB values in [0, 1], with some Gaussian structure
            genome = np.clip(
                np.random.randn(IMG_HEIGHT, IMG_WIDTH, CHANNELS) * 0.5 + 0.5,
                0, 1
            )
        self.genome = genome.astype(np.float32)
        self.fitness = 0.0

    def copy(self):
        clone = Individual(self.genome.copy())
        clone.fitness = self.fitness
        return clone


def mutate(genome, mutation_rate=MUTATION_RATE, mutation_std=MUTATION_STD):
    """
    Gaussian mutation applied to a random subset of pixels.
    """
    mask = np.random.rand(*genome.shape) < mutation_rate
    noise = np.random.randn(*genome.shape) * mutation_std
    mutated = genome.copy()
    mutated[mask] = mutated[mask] + noise[mask]
    mutated = np.clip(mutated, 0.0, 1.0)
    return mutated


def crossover(parent1, parent2):
    """
    Simple uniform crossover at the pixel level.
    """
    mask = np.random.rand(IMG_HEIGHT, IMG_WIDTH, CHANNELS) < 0.5
    child_genome = np.where(mask, parent1.genome, parent2.genome)
    return Individual(child_genome)


def init_population(size=POP_SIZE):
    return [Individual() for _ in range(size)]


def evolve_population(population, ratings):
    """
    Evolve a new population given current population and user ratings.

    population: list[Individual]
    ratings:    list[int] in [0, MAX_RATING]
    """
    # Assign fitness
    for ind, r in zip(population, ratings):
        ind.fitness = float(r)

    # Sort by fitness descending
    population = sorted(population, key=lambda ind: ind.fitness, reverse=True)

    # Elitism: keep the best individuals
    elites = [ind.copy() for ind in population[:ELITE_COUNT]]

    # If all ratings are zero, just re-randomize the population
    if max(ratings) == 0:
        return init_population(len(population)), elites

    # Selection probabilities proportional to (nonnegative) fitness
    fitnesses = np.array([max(ind.fitness, 0.0) for ind in population], dtype=np.float32)
    if fitnesses.sum() == 0:
        probs = np.ones_like(fitnesses) / len(fitnesses)
    else:
        probs = fitnesses / fitnesses.sum()

    def select_parent():
        idx = np.random.choice(len(population), p=probs)
        return population[idx]

    # Generate offspring to refill population
    new_population = elites.copy()
    while len(new_population) < len(population):
        p1 = select_parent()
        p2 = select_parent()
        child = crossover(p1, p2)
        child.genome = mutate(child.genome)
        new_population.append(child)

    return new_population, elites


# =======================
# Visualization Helpers
# =======================

def plot_individual(individual, ax):
    ax.imshow(individual.genome)
    ax.axis("off")


# =======================
# Interactive EA Class
# =======================

class InteractiveEA:
    def __init__(self):
        self.population = init_population()
        self.generation = 0
        self.history_best = []

        # Widgets
        self.out_images = Output()
        self.button_evolve = Button(
            description="Evolve to Next Generation",
            button_style="success",
            layout=Layout(width="250px")
        )
        self.button_randomize = Button(
            description="Randomize Population",
            button_style="warning",
            layout=Layout(width="200px")
        )
        self.info_label = HTML()
        self.rating_sliders = []

        # Wire up callbacks
        self.button_evolve.on_click(self.on_evolve_clicked)
        self.button_randomize.on_click(self.on_randomize_clicked)

        # Initial display
        self.update_display()

    def build_rating_widgets(self):
        self.rating_sliders = [
            IntSlider(
                value=0,
                min=0,
                max=MAX_RATING,
                step=1,
                description=f"#{i+1}",
                continuous_update=False,
                layout=Layout(width="220px")
            )
            for i in range(len(self.population))
        ]

    def update_display(self):
        # Rebuild sliders each generation
        self.build_rating_widgets()
        self.info_label.value = (
            f"<b>Generation:</b> {self.generation} "
            f"&nbsp;|&nbsp; Rate each image 0–{MAX_RATING}, then click <b>Evolve</b>."
        )

        # Clear the cell output so we don't stack UIs
        clear_output(wait=True)

        # Draw images
        with self.out_images:
            clear_output(wait=True)
            fig, axes = plt.subplots(3, 4, figsize=(8, 6))
            axes = axes.flatten()

            for ind, ax in zip(self.population, axes):
                plot_individual(ind, ax)

            # Hide unused axes (if any)
            for ax in axes[len(self.population):]:
                ax.axis("off")

            plt.tight_layout()
            plt.show()

        ratings_box = VBox(self.rating_sliders)
        controls = HBox([self.button_evolve, self.button_randomize])

        display(VBox([
            self.info_label,
            self.out_images,
            HTML("<b>Rate each image:</b>"),
            ratings_box,
            controls
        ]))

    def on_evolve_clicked(self, _):
        ratings = [slider.value for slider in self.rating_sliders]
        self.population, elites = evolve_population(self.population, ratings)
        best = max(ind.fitness for ind in self.population)
        self.history_best.append(best)
        self.generation += 1
        self.update_display()

    def on_randomize_clicked(self, _):
        self.population = init_population()
        self.generation = 0
        self.history_best = []
        self.update_display()


# =======================
# Launch the Interactive EA
# =======================

iea = InteractiveEA()
